In [1]:
from Densenet import *
from util import *

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt


n_classes_cifar10 = 10
train_size = 0.8
R = 5


# Download the entire CIFAR10 dataset

from torchvision.datasets import CIFAR10
import numpy as np 
from torch.utils.data import Subset
from torch.utils.data.sampler import SubsetRandomSampler


import torchvision.transforms as transforms

## Normalization is different when training from scratch and when training using an imagenet pretrained backbone

normalize_scratch = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))


# Data augmentation is needed in order to train from scratch
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize_scratch,
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_scratch,
])



### The data from CIFAR10 will be downloaded in the following dataset
rootdir = './data/cifar10'

c10train = CIFAR10(rootdir,train=True,download=True,transform=transform_train)
c10test = CIFAR10(rootdir,train=False,download=True,transform=transform_test)



# CIFAR10 is sufficiently large so that training a model up to the state of the art performance will take approximately 3 hours on the 1060 GPU available on your machine. 


def train_validation_split(train_size, num_train_examples):
    # obtain training indices that will be used for validation
    indices = list(range(num_train_examples))
    np.random.shuffle(indices)
    idx_split = int(np.floor(train_size * num_train_examples))
    train_index, valid_index = indices[:idx_split], indices[idx_split:]

    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_index)
    valid_sampler = SubsetRandomSampler(valid_index)

    return train_sampler,valid_sampler

def generate_subset(dataset,n_classes,reducefactor,n_ex_class_init):

    nb_examples_per_class = int(np.floor(n_ex_class_init / reducefactor))
    # Generate the indices. They are the same for each class, could easily be modified to have different ones. But be careful to keep the random seed! 

    indices_split = np.random.RandomState(seed=42).choice(n_ex_class_init,nb_examples_per_class,replace=False)

    all_indices = []
    for curclas in range(n_classes):
        curtargets = np.where(np.array(dataset.targets) == curclas)
        indices_curclas = curtargets[0]
        indices_subset = indices_curclas[indices_split]
        #print(len(indices_subset))
        all_indices.append(indices_subset)
    all_indices = np.hstack(all_indices)
    
    return Subset(dataset,indices=all_indices)
    


### These dataloader are ready to be used to train for scratch 
cifar10_train= generate_subset(dataset=c10train,n_classes=n_classes_cifar10,reducefactor=R,n_ex_class_init=5000)
num_train_examples=len(cifar10_train)
train_sampler,valid_sampler=train_validation_split(train_size, num_train_examples)

cifar10_test = generate_subset(dataset=c10test,n_classes=n_classes_cifar10,reducefactor=1,n_ex_class_init=1000) 



from torch.utils.data.dataloader import DataLoader

trainloader = DataLoader(c10train,batch_size=32,sampler=train_sampler)
validloader = DataLoader(c10train,batch_size=32,sampler=valid_sampler)
testloader = DataLoader(c10test,batch_size=32) 


def evaluation(model, test_loader, criterion): 
    class_names = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    

    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    model.eval()
    for data, label in test_loader:
        data = data.to(device=device, dtype=torch.float32)
        label = label.to(device=device, dtype=torch.long)
        with torch.no_grad():
            output = model(data)
        loss = criterion(output, label)
        test_loss += loss.item()*data.size(0)
        _, pred = torch.max(output, 1)
        correct = np.squeeze(pred.eq(label.data.view_as(pred)))
        for i in range(len(label)):
            digit = label.data[i]
            class_correct[digit] += correct[i].item()
            class_total[digit] += 1

    test_loss = test_loss/len(test_loader.sampler)
    print('test Loss: {:.6f}\n'.format(test_loss))
    for i in range(10):

        if(np.sum(class_total[i])==0):
            print(class_names[i])
        else:
            print('test accuracy of %s: %2d%% (%2d/%2d)' % (class_names[i], 100 * class_correct[i] / class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])))
    print('\ntest accuracy (overall): %2.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total), np.sum(class_correct), np.sum(class_total)))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#from minicifar import minicifar_train,minicifar_test,train_sampler,valid_sampler
from torch.utils.data.dataloader import DataLoader

trainloader = DataLoader(c10train,batch_size=128,sampler=train_sampler)
validloader = DataLoader(c10train,batch_size=128,sampler=valid_sampler)
testloader = DataLoader(c10test,batch_size=128) 

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device '+str(device))

Using device cpu


In [12]:
import os
import torch
import torch.nn.utils.prune as prune
import torch.nn as nn

def count_conv2d(m, x, y):
    x = x[0] # remove tuple

    fin = m.in_channels
    fout = m.out_channels
    sh, sw = m.kernel_size

    # ops per output element
    kernel_mul = sh * sw * fin
    kernel_add = sh * sw * fin - 1
    bias_ops = 1 if m.bias is not None else 0
    kernel_mul = kernel_mul/8 # FP4
    ops = kernel_mul + kernel_add + bias_ops

    # total ops
    num_out_elements = y.numel()
    total_ops = num_out_elements * ops
    
    # total params 
    

    print("Conv2d: S_c={}, F_in={}, F_out={}, P={}, params={}, operations={}".format(sh,fin,fout,x.size()[2:].numel(),int(m.total_params.item()),int(total_ops)))
    # incase same conv is used multiple times
    m.total_ops += torch.Tensor([int(total_ops)])


def count_bn2d(m, x, y):
    x = x[0] # remove tuple

    nelements = x.numel()
    total_sub = 2*nelements
    total_div = nelements
    total_ops = total_sub + total_div

    m.total_ops += torch.Tensor([int(total_ops)])
    print("Batch norm: F_in={} P={}, params={}, operations={}".format(x.size(1),x.size()[2:].numel(),int(m.total_params.item()),int(total_ops)))


def count_relu(m, x, y):
    x = x[0]

    nelements = x.numel()
    total_ops = nelements

    m.total_ops += torch.Tensor([int(total_ops)])
    print("ReLU: F_in={} P={}, params={}, operations={}".format(x.size(1),x.size()[2:].numel(),0,int(total_ops)))



def count_avgpool(m, x, y):
    x = x[0]
    total_add = torch.prod(torch.Tensor([m.kernel_size])) - 1
    total_div = 1
    kernel_ops = total_add + total_div
    num_elements = y.numel()
    total_ops = kernel_ops * num_elements

    m.total_ops += torch.Tensor([int(total_ops)])
    print("AvgPool: S={}, F_in={}, P={}, params={}, operations={}".format(m.kernel_size,x.size(1),x.size()[2:].numel(),0,int(total_ops)))

def count_linear(m, x, y):
    # per output element
    total_mul = m.in_features/2
    total_add = m.in_features - 1
    num_elements = y.numel()
    total_ops = (total_mul + total_add) * num_elements
    print("Linear: F_in={}, F_out={}, params={}, operations={}".format(m.in_features,m.out_features,int(m.total_params.item()),int(total_ops)))
    m.total_ops += torch.Tensor([int(total_ops)])

def count_sequential(m, x, y):
    print ("Sequential: No additional parameters  / op")

# custom ops could be used to pass variable customized ratios for quantization
def profile(model, input_size, custom_ops = {}):

    model.eval()

    def add_hooks(m):
        if len(list(m.children())) > 0: return
        m.register_buffer('total_ops', torch.zeros(1))
        m.register_buffer('total_params', torch.zeros(1))


        #for p in m.parameters():
         #   m.total_params += torch.Tensor([torch.count_nonzero(p.data)]) / 8
            


        if isinstance(m, nn.Conv2d):
            m.register_forward_hook(count_conv2d)
        elif isinstance(m, nn.BatchNorm2d):
            m.register_forward_hook(count_bn2d)
        elif isinstance(m, nn.ReLU):
            m.register_forward_hook(count_relu)
        elif isinstance(m, (nn.AvgPool2d)):
            m.register_forward_hook(count_avgpool)
        elif isinstance(m, nn.Linear):
            m.register_forward_hook(count_linear)
        elif isinstance(m, nn.Sequential):
            m.register_forward_hook(count_sequential)
        else:
            print("Not implemented for ", m)

    model.apply(add_hooks)

    x = torch.zeros(input_size)
    model(x)

    total_ops = 0
    total_params = 0
    for m in model.modules():
        if len(list(m.children())) > 0: continue
        total_ops += m.total_ops
        total_params += m.total_params

    return total_ops, total_params

def score(model , filename, pruning = False, quantization = False):
    ref_params = 5586981
    ref_flops  = 834362880
    if(pruning):
        criterion = nn.CrossEntropyLoss()
        parameters_to_prune=[]
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear) :
                parameters_to_prune.append((module,'weight'))
        prune.global_unstructured(parameters_to_prune, pruning_method=prune.L1Unstructured, amount=0.2)
        loaded_cpt=torch.load(filename)
        model.load_state_dict(loaded_cpt)
        evaluation(model, testloader, criterion)
        
    flops, _ = profile(model, (1,3,32,32))
    flops = flops.item()
    
    params = 0
    if(quantization):
        for name, para in model.named_parameters():
            if "conv" in name and name.startswith("conv") == False:
                params+= para.nonzero().size(0)/8
            else:
                params+= para.nonzero().size(0)/2
    else:
        for name, para in model.named_parameters():
            params+= para.nonzero().size(0)/2
    return flops/ref_flops , params/ref_params

        

def main():

    ref_params = 5586981
    ref_flops  = 834362880

    import torch.nn.utils.prune as prune
    model = densenet_cifar()
    
    model.to(device=device)
    
    criterion = nn.CrossEntropyLoss()

    evaluation(model, testloader, criterion)
    
    parameters_to_prune=[]
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear) :
            parameters_to_prune.append((module,'weight'))
    prune.global_unstructured(parameters_to_prune, pruning_method=prune.L1Unstructured, amount=0.2)
    loaded_cpt=torch.load('densenet_cifar_10_pruning_globally_0.06.pt')
    model.load_state_dict(loaded_cpt)
    
    

    evaluation(model, testloader, criterion)

    
    from torchvision import models


    model.to(device=device)

    
    evaluation(model,testloader, criterion)
    
    flops, params = profile(model, (1,3,32,32))
    flops, params = flops.item(), params.item()
    
    params = 0
    for name, para in model.named_parameters():
        if "conv" in name:
            params+= para.nonzero().size(0)/8
        else:
            params+= para.nonzero().size(0)/2
    
    print(sum(p.numel() for p in model.parameters()))
    
    score_flops = flops / ref_flops
    score_params = params / ref_params
    score = score_flops + score_params
    print("Flops: {}, Params: {}".format(flops,params))
    print("Score flops: {} Score Params: {}".format(score_flops,score_params))
    print("Final score: {}".format(score))

In [14]:
model = densenet_cifar()
print(score(model,"",quantization = True))

Conv2d: S_c=3, F_in=3, F_out=24, P=1024, params=0, operations=721920
Batch norm: F_in=24 P=1024, params=0, operations=73728
Conv2d: S_c=1, F_in=24, F_out=48, P=1024, params=0, operations=1277952
Batch norm: F_in=48 P=1024, params=0, operations=147456
Conv2d: S_c=3, F_in=48, F_out=12, P=1024, params=0, operations=5959680
Batch norm: F_in=36 P=1024, params=0, operations=110592
Conv2d: S_c=1, F_in=36, F_out=48, P=1024, params=0, operations=1941504
Batch norm: F_in=48 P=1024, params=0, operations=147456
Conv2d: S_c=3, F_in=48, F_out=12, P=1024, params=0, operations=5959680
Batch norm: F_in=48 P=1024, params=0, operations=147456
Conv2d: S_c=1, F_in=48, F_out=48, P=1024, params=0, operations=2605056
Batch norm: F_in=48 P=1024, params=0, operations=147456
Conv2d: S_c=3, F_in=48, F_out=12, P=1024, params=0, operations=5959680
Batch norm: F_in=60 P=1024, params=0, operations=184320
Conv2d: S_c=1, F_in=60, F_out=48, P=1024, params=0, operations=3268608
Batch norm: F_in=48 P=1024, params=0, opera

In [21]:
main()

test Loss: 2.303173

test accuracy of plane:  0% ( 0/1000)
test accuracy of car:  0% ( 0/1000)
test accuracy of bird:  0% ( 0/1000)
test accuracy of cat:  0% ( 0/1000)
test accuracy of deer:  0% ( 0/1000)
test accuracy of dog:  0% ( 0/1000)
test accuracy of frog:  0% ( 0/1000)
test accuracy of horse:  0% ( 0/1000)
test accuracy of ship:  0% ( 0/1000)
test accuracy of truck: 100% (1000/1000)

test accuracy (overall): 10.00% (1000/10000)
test Loss: 0.292871

test accuracy of plane: 92% (923/1000)
test accuracy of car: 96% (967/1000)
test accuracy of bird: 89% (892/1000)
test accuracy of cat: 83% (830/1000)
test accuracy of deer: 93% (933/1000)
test accuracy of dog: 86% (867/1000)
test accuracy of frog: 95% (951/1000)
test accuracy of horse: 93% (938/1000)
test accuracy of ship: 95% (953/1000)
test accuracy of truck: 94% (945/1000)

test accuracy (overall): 91.99% (9199/10000)
test Loss: 0.292871

test accuracy of plane: 92% (923/1000)
test accuracy of car: 96% (967/1000)
test accuracy of